In [ ]:
from ultralytics import YOLO
import torch

In [ ]:
"""
This Notebook will be tha base for all the YOLO training that Minna does in the project.
"""

In [ ]:
if torch.cuda.is_available():
    device = 0
    print(f"Training on GPU")
else:
    print(f"Training on CPU")
    device = "cpu"

In [ ]:
"""
Default values used for all YOLO trainings.

These parameters should remain consistent across different training runs,
unless the hardware or training context changes significantly.
"""

model = YOLO("yolo11s.pt")

default_args = {
    "data": "config/materials.yaml",     # dataset configuration path
    "project": "material_koulutus",      # output base folder
    "imgsz": 768,                     # input size for YOLO
    "batch": 16,                      # batch size (GPU memory limit)
    "device": 0,                      # GPU
    "cache": "disk",                  # disk is better that ram
    "workers": 4,                     # parallel data loader workers
    "amp": True,                      # automatic mixed precision (faster training)
    "mosaic": 0.2,
    "save": True,                     # save checkpoints and weights
    "val": True,                      # enable validation after each epoch
    "plots": True,                    # save training/validation plots
    "patience": 10                    # stop training if results don´t improve
}

In [ ]:
"""
Run-specific training parameters.

These parameters can be adjusted per experiment to test different configurations,
optimizers, learning rates, or other hyperparameters.
"""

results = model.train(
    **default_args,
    epochs=80,                        # number of training epochs
    optimizer="Adam",                 # optimizer: AdamW, Adam, or SGD
    lr0=0.0003,                       # initial learning rate
    momentum=0.9,                     # used with optimizers
    weight_decay=0.0005,              # regularization to prevent overfitting
    warmup_epochs=5.0,                # warm-up period for learning rate
    label_smoothing=0.1,
    cos_lr=True,                      # can improve last epochs
    name="train_minna_p5"             # unique name for this training run
)

In [ ]:
"""
This script reads YOLO training results from a CSV file and visualizes key evaluation metrics:
- Training and validation losses
"""

%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

csv_path = "material_koulutus/train_minna_p5/results.csv"
df = pd.read_csv(csv_path)
df.columns = df.columns.str.strip()

display(df.head())
display(df.tail())

df[["train/box_loss", "train/cls_loss", "train/dfl_loss"]].plot(figsize=(10, 5), title="Training Losses per Epoch")
plt.xlabel("Epoch")
plt.grid()
plt.show()

model = YOLO("material_koulutus/train_minna_p5/weights/best.pt")
results = model.val(conf=0.3)